<a href="https://colab.research.google.com/github/elvisbui/Predicting-Length-of-Stay/blob/master/EDA_and_Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Exploratory Data Analysis

In this notebook, I will be doing a basic analysis of healthcare data and creating a baseline model from this dataset: https://www.kaggle.com/nehaprabhavalkar/av-healthcare-analytics-ii

I will start with a simple pipeline, check that it runs properly, and slowly add more complexity to it. This way of program makes it easy to debug and fix errors. The baseline model will give me insight on how other models compare and how to improve my models. 

### Load Libraries 
The first step is to load the libraries we will be using. 

In [ ]:
import math

# import data processing and linear algebra libraries 
import pandas as pd
import numpy as np


# import data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns


from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix

# Setting seeds to remove randomness 
RANDOM_STATE = 24

### Load Data
Next is getting the data and loading it. 

In [ ]:
TRAIN_DIR = '../input/av-healthcare-analytics-ii/healthcare/train_data.csv'
TEST_DIR = '../input/av-healthcare-analytics-ii/healthcare/test_data.csv'
SAMPLE_SUBM = '../input/av-healthcare-analytics-ii/healthcare/sample_sub.csv'
TRAIN_DICT_DIR = '../input/av-healthcare-analytics-ii/healthcare/train_data_dictionary.csv'

def read_csv(*paths: str) -> tuple:
    '''
    Gets a list of cvs paths and returns all cvs in a tuple

            Parameters:
                    *paths (tuple of str): A decimal integer

            Returns:
                    binary_sum (tuple of dataframe): tuple of cvs dataframes
    '''
    result = []
    for dir in paths:
        csv = pd.read_csv(dir)
        result.append(csv)
    return tuple(result)

train, test, sample_subm, train_dict = read_csv(TRAIN_DIR, TEST_DIR, SAMPLE_SUBM, TRAIN_DICT_DIR)

# Looking at the data
Let's not take a peek at the data.

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.dtypes

By looking at the data types of each column, I see that we are dealing with a lot of categorical data.
This leads me to think that CatBoost might be well suited for this data. Let's see how many unique values each column has. 

In [ ]:
train.nunique()

Let's now try to understand which categorical data is nominal, and which is ordinal.

In [ ]:
train_dict

In [ ]:
train.Hospital_code.unique()

While the hospital code are numerical, I do not believe there is a ranking to them. A hospital with a top 1 does not mean it is better than a hospital with a code 20, and vice versa. 

Let's keep checking the other columns. 
Hospital_type_code, City_Code_Hospital, Hospital_region_code, Department, Ward_Facility_Code, and Ward_Type are all nominal data. 

In [ ]:
train['Type of Admission'].unique()

In [ ]:
train['Severity of Illness'].unique()

In [ ]:
train['Bed Grade'].unique()

Type of Admission, Severity of Illness, and Bed Grade are all ordinal data. This will help us decide on which encoding to use for which feature. 

# Checking for missing values

In [ ]:
train.isnull().sum()

It looks like only bed grade and city code patient have missing values. We could fill in the missing bed grade values with the median of all bed grades. Since there are return patients, we can try to get the missing city code patient values from other rows with the same patient id. And if there are no other rows with the same patient id, we can try to find rows with similiar data and use their city patient code to replace the missing values. 

We can also use a boost tree since they deal well with missing values. 

# Check for outliers

In [ ]:
train.describe()

Extra rooms, visitors, and admission deposit seems like they might have outliers. The problem with determine outliers is that we need to determine if they are accutally outliers or just extreme entries that need to be taken into consideration for our models. 

In [ ]:
train.boxplot('Available Extra Rooms in Hospital')

In [ ]:
train.boxplot('Visitors with Patient')

In [ ]:
train.boxplot('Admission_Deposit')

Dealing with outliers is tricky because outliers might help improve the performance of our models. Looking at the boxplot, it seems like 'Available Extra Rooms in Hospital' have few outliers, but we should not remove them from the data because available extra rooms change a lot in the real world. The same can be said for visitors and admsision deposit. 

Let's check out some correlations. 

In [ ]:
def graph_groupby_values(df, column1, column2):
    col1_names = df[column1].unique().tolist()
    col1_names.sort()
    fig, axes = plt.subplots(nrows=math.ceil(len(col1_names)/2), ncols=2)
    for idx, name in enumerate(col1_names):
        gb = train.loc[train[column1] == name].groupby(column2)
        gb.size().plot(kind='bar', ax=axes[idx//2, idx%2], figsize=(14, 25), title=column1+': '+ name)
    fig.tight_layout(pad=3.0)
        
graph_groupby_values(train, 'Age', 'Stay')

# Preprocessing
Let's label encode some of the features for Catboost. 

In [ ]:
train.head()

We can use Pandas's factorize and scikit-learn's LabelEncoder functions for label encoding, but for ordinal features like 'Stay' and 'Type of Admission', we need more control over the mapping. And since we are using a tree boost, we do not need to scale the data.

In [ ]:
stay_encode = {'0-10': 0,
               '11-20': 1,
               '21-30': 2,
               '31-40': 3,
               '41-50': 4,
               '51-60': 5,
               '61-70': 6,
               '71-80': 7,
               '81-90': 8,
               '91-100': 9,
               'More than 100 Days': 10}

hospital_type_code_encode={'a': 0,
                           'b': 1,
                           'c': 2,
                           'e': 3,
                           'd': 4,
                           'f': 5,
                           'g': 6} 

hospital_region_encode = {'X': 0, 
                          'Y': 1, 
                          'Z': 2}

department_encode={'anesthesia': 0,
                   'gynecology': 1,
                   'radiotherapy': 2,
                   'surgery': 3,
                   'TB & Chest disease': 4,}

word_type_encode = {'P': 0,
                    'Q': 1,
                    'R': 2,
                    'S': 3,
                    'T': 4,
                    'U': 5}

ward_facility_code_encode ={'A': 0, 
                            'B': 1, 
                            'C': 2, 
                            'D': 3, 
                            'E': 4, 
                            'F': 5}

admission_type_encode = {'Trauma': 0, 
                         'Emergency': 1, 
                         'Urgent': 2}

illness_encode = {'Minor': 0,
                  'Moderate ': 1,
                  'Extreme': 2}

age_encode = {'0-10': 0,
              '11-20': 1,
              '21-30': 2,
              '31-40': 3,
              '41-50': 4,
              '51-60': 5,
              '61-70': 6,
              '71-80': 7,
              '81-90': 8,
              '91-100': 9}

In [ ]:
train['Stay'] = train['Stay'].map(stay_encode)
train['Hospital_type_code'] = train['Hospital_type_code'].map(hospital_type_code_encode)
train['Hospital_region_code'] = train['Hospital_region_code'].map(hospital_region_encode)
train['Department'] = train['Department'].map(department_encode)
train['Ward_Type'] = train['Ward_Type'].map(word_type_encode)
train['Ward_Facility_Code'] = train['Ward_Facility_Code'].map(ward_facility_code_encode)
train['Type of Admission'] = train['Type of Admission'].map(admission_type_encode)
train['Severity of Illness'] = train['Severity of Illness'].map(illness_encode)
train['Age'] = train['Age'].map(age_encode)

In [ ]:
train.head()

Now that we have our data encoded, let's check the correlations. 

In [ ]:
import seaborn as sns
corr = train.corr()
sns.heatmap(corr, 
            xticklabels=train.columns.values,
            yticklabels=train.columns.values)

From the heatmap, I can see that amount of visitors, age, and severity of illness has a relatively high correlation with stay length. We will use this insight when we create new features. 

# Let's now create a baseline model

We need to seperate the target column from the training data. And then we need to split the data for training and for testing. We are not using k fold cross validation in this basemodel because we just want to get the baseline model to run and get insight from the results. 

For catboost, we need the indices of categorical features. Patient ID might seem numerical at first, but since there is no ranking between the the different ID numbers, the feature is nominal and so it is categorical. 

In [ ]:
cat_features = [0,1,2,3,5,6,7,9]

y = train.loc[:,['Stay']]
X = train.drop(['Stay','case_id'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=RANDOM_STATE)

In [ ]:
X.iloc[:,cat_features]

In [ ]:
clf = CatBoostClassifier(task_type="GPU", custom_metric='Accuracy', eval_metric='Accuracy', random_seed=RANDOM_STATE)
history = clf.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_val, y_val), plot=True, early_stopping_rounds=100)

# Review Model Score

In [ ]:
train_score = clf.score(X_train, y_train) # train (learn) score
val_score = clf.score(X_val, y_val) # val (test) score

In [ ]:
print('Train Score:', train_score)
print('Validation Score:', val_score)

We can see that our training score is a little higher than our validation score. This could mean that the model is overfitting the training data a little. Let's take a closer look at the results of our model. 

In [ ]:
def report_model_score(model, X, y):
    y_pred = model.predict(X)
    corr = confusion_matrix(y, y_pred)
    print('Confusion Matrix: \n', corr)
    print('\nAccuracy Score: \n', accuracy_score(y, y_pred))
    # print('\nROC AUC: \n', roc_auc_score(y, y_pred))
    print('\nReport: \n', classification_report(y,y_pred))
    y_labels = range(0,11)
    sns.heatmap(corr, xticklabels=y_labels, yticklabels=y_labels)
    # plot_confusion_matrix(model, X, y)
    # plt.show() 

In [ ]:
report_model_score(clf, X_val, y_val)

# Error Analysis

We see that our model can predict the length of stay at 1, 2, 3, and 5 well. What is strange is that the length of stay at 4 is getting predict to be 2. Stay in group 4 does poorly overall. 